In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

T_f = 350.0

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSD%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFNSD%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag, init_values=T_bot)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=T_f)
tempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 2000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FNSD%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Dirichlet setting, study case: %i\n' % k)
flog.close()
flog = open('FNSD%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    # mapping
    fluxF.curr[:] = fluxS.curr[s2f]
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=fluxF.curr
    )
    
    while True:
        fluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface temperature
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        tempS.curr[:] = tempF.curr[f2s]
        
        solverS.update_temperature_interface(interface_tag, tempS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        fluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        fluxF.curr[:] = fluxS.curr[s2f]
        
        fluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to solid then restore
            if use_relax:
                under_relax.determine_omega(fluxF)
                under_relax.update_solution(fluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=fluxF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=20.
step=1, pc_iterations=17.
step=2, pc_iterations=14.
step=3, pc_iterations=16.
step=4, pc_iterations=15.
step=5, pc_iterations=16.
step=6, pc_iterations=14.
step=7, pc_iterations=16.
step=8, pc_iterations=15.
step=9, pc_iterations=15.
step=10, pc_iterations=15.
step=11, pc_iterations=14.
step=12, pc_iterations=14.
step=13, pc_iterations=14.
step=14, pc_iterations=14.
step=15, pc_iterations=14.
step=16, pc_iterations=14.
step=17, pc_iterations=14.
step=18, pc_iterations=13.
step=19, pc_iterations=15.
step=20, pc_iterations=14.
step=21, pc_iterations=15.
step=22, pc_iterations=14.
step=23, pc_iterations=15.
step=24, pc_iterations=14.
step=25, pc_iterations=15.
step=26, pc_iterations=14.
step=27, pc_iterations=15.
step=28, pc_iterations=14.
step=29, pc_iterations=14.
step=30, pc_iterations=14.
step=31, pc_iterations=14.
step=32, pc_iterations=14.
step=33, pc_iterations=14.
step=34, pc_iterations=14.
step=35, pc_iterations=14.
step=36, pc_iterations=14.
step=37, pc

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
print(theta)
sio.savemat(
    'numeric_res/kFNSD%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.0770442  0.07066682 0.06555317 0.06264919 0.06017282 0.05788897
 0.05586439 0.05402239 0.05224435 0.05065818 0.0491631  0.04777433
 0.04642406 0.04516546 0.0439619  0.04283606 0.04173856 0.04070396
 0.03972197 0.03876613 0.03786273 0.03698957 0.03615355 0.03535819
 0.0345848  0.03384503 0.03312678 0.03245089 0.03180908 0.03118041
 0.03057194 0.02999662 0.02943428 0.02889361 0.0283765  0.0278813
 0.02739861 0.02693623 0.02648593 0.02606092 0.02565329 0.02526237
 0.024881   0.0245144  0.02416239 0.02382446 0.02349756 0.02318739
 0.02289014 0.02259871 0.02232084 0.02205822 0.02180482 0.0215621
 0.02133427 0.02111092 0.02090455 0.020704   0.02051349 0.02033629
 0.02017023 0.02000942 0.01986009 0.01972352 0.01959643 0.01947634
 0.01937311 0.01927375 0.0191929  0.01911952 0.01904995 0.01899903
 0.01896173 0.01893024 0.01891319 0.01891003 0.01892024 0.01894184
 0.01898365 0.0190333  0.01910624 0.01919186 0.01929284 0.01941303
 0.01955566 0.01972363 0.01990558 0.02012013 0.0203646  0.020633